[Reference](https://towardsdatascience.com/automate-email-with-python-1e755d9c6276)

# MIME

In [2]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email.mime.audio import MIMEAudio

msg = MIMEMultipart  # initialize the MIME object, containing our email message
msg.attach(MIMEText(text))  # add text contents

img_data = open('image.jpg', 'rb').read()  # read an image binary data
msg.attach(MIMEImage(img_data))  # add the image to our message object

# read in attachment as binary
with open('report.docx', 'rb') as f:
    file = MIMEApplication(f.read())  # read the attachment file
msg.attach(file)  # add the attachment to our message object

In [1]:
import os
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart

def message(subject="Python Notification", text="", img=None, attachment=None):
    # build message contents
    msg = MIMEMultipart()
    msg['Subject'] = subject  # add in the subject
    msg.attach(MIMEText(text))  # add text contents

    # check if we have anything given in the img parameter
    if img is not None:
        # if we do, we want to iterate through the images, so let's check that
        # what we have is actually a list
        if type(img) is not list:
            img = [img]  # if it isn't a list, make it one
        # now iterate through our list
        for one_img in img:
            img_data = open(one_img, 'rb').read()  # read the image binary data
            # attach the image data to MIMEMultipart using MIMEImage, we add
            # the given filename use os.basename
            msg.attach(MIMEImage(img_data, name=os.path.basename(one_img)))

    # we do the same for attachments as we did for images
    if attachment is not None:
        if type(attachment) is not list:
            attachment = [attachment]  # if it isn't a list, make it one
            
        for one_attachment in attachment:
            with open(one_attachment, 'rb') as f:
                # read in the attachment using MIMEApplication
                file = MIMEApplication(
                    f.read(),
                    name=os.path.basename(one_attachment)
                )
            # here we edit the attached file metadata
            file['Content-Disposition'] = f'attachment; filename="{os.path.basename(one_attachment)}"'
            msg.attach(file)  # finally, add the attachment to our message object
    return msg

# SMTP (Simple Mail Transfer Protocol)

![smtp](https://miro.medium.com/max/1400/1*tGCyXhXldhyxxGUsh9Yw6w.png)

In [ ]:
import smtplib

# initialize connection to our email server, we will use Outlook here
smtp = smtplib.SMTP('smtp-mail.outlook.com', port='587')

smtp.ehlo()  # send the extended hello to our server
smtp.starttls()  # tell server we want to communicate with TLS encryption

smtp.login('joe.bloggs@outlook.com', 'Password123')  # login to our email server

# send our email message 'msg' to our boss
smtp.sendmail('joe.bloggs@outlook.com',
              'joes.boss@outlook.com',
              msg.as_string())
              
smtp.quit()  # finally, don't forget to close the connection